## Client Collaboration
First, let's set up our client. Fill in the same username you used in the previous notebook to generate your certificate.

In [1]:
import securexgboost as mc2
from Utils import * 

In [2]:
# TODO: fill in the IP of the enclave server
server_ip = "13.68.246.44"
server_port = "50052" 
remote_addr = server_ip + ":" + server_port 

In [3]:
# Get variables from previous notebooks
%store -r

## Data Transfer
Next, centralize the data by send your training and test data to the person controlling the enclave server. Don't worry, though; your data is encrypted, and the controlling party won't be able to see your mushroom data. 

The `transfer_data()` function below is a Python wrapper around the command line `scp` function.

In [4]:
transfer_data(enc_training_data, server_ip)
transfer_data(enc_test_data, server_ip)

Transferring /home/mc2/risecamp/mc2/tutorial/data/chief_train.enc to 13.68.246.44
Transferring /home/mc2/risecamp/mc2/tutorial/data/chief_test.enc to 13.68.246.44


## Client Initialization and Authentication
Once everyone has sent their data to the server, we can initialize our client. You'll need the usernames of all mushroom enthuasists in your collaboration.

**TODO:** Fill in the usernames of all parties in your collaboration.

In [7]:
# Run this cell to initialize your client

# TODO: fill out `clients`

###########################################################
# `clients` is a Python list, e.g.
#
# clients = ["alice", "bob"]
###########################################################
clients = ['chief']
mc2.init_client(user_name=username, client_list=clients, sym_key_file=KEY_FILE, 
                priv_key_file=PUB_KEY, cert_file=CERT_FILE, remote_addr=remote_addr)

Before we perform any computation, we want to attest that the remote enclave on the untrusted server has loaded the proper code. Secure XGBoost provides this functionality through the `attest()` API.

In [8]:
# Verify that the enclave has been set up correctly
mc2.attest()

XGBoostError: [20:48:27] /home/mc2/secure-xgboost/host/src/c_api/c_api_mc.cc:705: Remote attestation failed. SHA256 mismatch.
Stack trace:
  [bt] (0) /usr/local/lib/python3.6/dist-packages/securexgboost-0.1-py3.6.egg/securexgboost/libxgboost.so(_ZN4dmlc15LogMessageFatalD1Ev+0x7c) [0x7fa5d9c51cbc]
  [bt] (1) /usr/local/lib/python3.6/dist-packages/securexgboost-0.1-py3.6.egg/securexgboost/libxgboost.so(_Z20attest_remote_reportPKhmS0_m+0x197) [0x7fa5d9c46dd7]
  [bt] (2) /usr/local/lib/python3.6/dist-packages/securexgboost-0.1-py3.6.egg/securexgboost/libxgboost.so(verify_remote_report_and_set_pubkey_and_nonce+0x335) [0x7fa5d9c47315]
  [bt] (3) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call_unix64+0x4c) [0x7fa61982adae]
  [bt] (4) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call+0x22f) [0x7fa61982a71f]
  [bt] (5) /usr/lib/python3.6/lib-dynload/_ctypes.cpython-36m-x86_64-linux-gnu.so(_ctypes_callproc+0x2b4) [0x7fa619a3e5c4]
  [bt] (6) /usr/lib/python3.6/lib-dynload/_ctypes.cpython-36m-x86_64-l

## Collaborative Training
Once we've authenticated the enclave, we can begin making requests to the enclave server. MC<sup>2</sup> enables users to make requests through a Python API, but will only execute requests once all users in the collaboration have submitted the same request. Consequently, users must submit the exact same requests in the exact order if they want to collaboratively compute. 

In particular, if you submit a request, the RPC orchestrator will queue up your request and only relay the request to the enclave server once all members of the collaboration have submitted the same request. Consequently, the execution of a cell containing a MC<sup>2</sup> API call will only finish once all parties have called the same function and the enclave server has returned from that function.

Let's first prepare for training by loading everyone's encrypted training data within the enclave. 

Fill in the paths to each party's training data. Your training data is at `<your_username>_train.enc`.

MC<sup>2</sup>'s `DMatrix()` function takes in a dictionary:

`{"username1": "user1.data", "username2": "user2.data"}`. 

In [ ]:
# TODO: fill in usernames and training data paths

#####################################################################################################################
# For example if the collaboration has two users, `alice` and `bob`, 
# the following would look like
#
# dtrain = mc2.DMatrix({"alice": "alice_train.enc", "bob": "bob_train.enc"})
#####################################################################################################################

dtrain = mc2.DMatrix({<****>: <****>, 
                      <****>: <****>,
                      <****>: <****>, 
                      <****>: <****>})

Next, jointly train a model over all mushroom samples shared by your group!

In [ ]:
# Set parameters
params = {
        "tree_method": "hist",
        "objective": "binary:logistic",
        "min_child_weight": "1",
        "gamma": "0.1",
        "max_depth": "3",
        "verbosity": "1" 
}

num_rounds = 10
booster = mc2.train(params, dtrain, num_rounds)

## Prediction Serving
Once we've jointly trained a model, we'll use the model to serve predictions on each party's test data. Each party should load its data into a separate object so that the model will output a set of predictions on only that party's test data. Predictions served by MC<sup>2</sup> are encrypted and can only be decrypted by the owner of the test data.

Remember that a request can only be executed if every party allows it. As a result, we'll need to submit a request to load test data for _every party_. 

**TODO:** Fill in usernames and paths to test data for each user. Your test data is at path `<your_username>_test.enc`.

In [ ]:
# Load everyone's test data

# TODO: fill in usernames and test data paths
dtest1 = mc2.DMatrix({<****>: <****>})
dtest2 = mc2.DMatrix({<****>: <****>})
dtest3 = mc2.DMatrix({<****>: <****>})
dtest4 = mc2.DMatrix({<****>: <****>})

Once we've loaded each party's test data, we'll need to ask MC<sup>2</sup> to serve predictions on each set of test data. However, we only want (and will only be able to decrypt) the predictions on our test data, so only store the return value of the `predict()` function called on our test data.

The `predict()` function returns two values: `(encrypted_predictions, num_predictions)`.

In [ ]:
# Allow everyone to get their predictions

##################################################################################################
# Remember to store the predictions on your test data. For example, if you own `dtest3`, your code
# should look like the following
#
#   booster.predict(dtest1)
#   booster.predict(dtest2)
#   enc_preds, num_preds = booster.predict(dtest3)
#   booster.predict(dtest4)
##################################################################################################

booster.predict(dtest1)
booster.predict(dtest2)
booster.predict(dtest3)
booster.predict(dtest4)

At this point, each party has obtained a set of encrypted predictions. They now have a better idea of whether their mysterious mushroom samples are edible! Decrypt the predictions to reap the benefits of the collaboration and of being a member of the mushroom enthuasist group.

In [ ]:
# Decrypt our predictions
preds = booster.decrypt_predictions(enc_preds, num_preds)
print(preds[:10])